In [1]:
# LTN: https://github.com/logictensornetworks/logictensornetworks/blob/master/examples/multiclass_classification/multiclass-singlelabel.ipynb
# Common.py : https://github.com/logictensornetworks/logictensornetworks/raw/master/examples/multiclass_classification/commons.py

In [2]:
!pip install PyTDC rdkit-pypi ltn keras==2.15.0 -qq
!wget https://github.com/logictensornetworks/logictensornetworks/raw/master/examples/multiclass_classification/commons.py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.3/151.3 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import logging; logging.basicConfig(level=logging.INFO)
import tensorflow as tf
import pandas as pd
import numpy as np
import ltn
from rdkit import Chem
from rdkit.Chem import AllChem
from tqdm.auto import tqdm
tqdm.pandas()

In [5]:
# Threshold ref: https://pubs.acs.org/doi/epdf/10.1021/acs.jcim.3c01301
def label_th(pic50):
    classes = []
    for x in pic50:
        if x>=5:
            classes.append(1)
        else:
            classes.append(0)

    return np.asarray(classes)

class_map = {
    "blocks":1,
    "non-blocks":0,
}

### Data Acquisition

In [6]:
dataset_path = "/content/drive/MyDrive/Project/AI and Cardiology/Cardiotoxicity/Dataset"

In [7]:
!ls "{dataset_path}/UniChemDB-Data"

CDK-unichemdb.csv     herg-gemini-embedding.parquet  standized-herg.csv
final-herg.csv	      herg_uniherg_db_mmb_emb.npy    uniherg_db-deepseek-qwen1_5b-embedding.parquet
final-herg-split.csv  Morgan-unichemdb.csv


In [8]:
df = pd.read_parquet(f"{dataset_path}/UniChemDB-Data/herg-gemini-embedding.parquet")

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20409 entries, 0 to 20408
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                20388 non-null  object
 1   std_smiles        20409 non-null  object
 2   classes           20409 non-null  int64 
 3   train_test_split  20409 non-null  int64 
 4   cv_fold           20409 non-null  int64 
 5   Embeddings        20409 non-null  object
dtypes: int64(3), object(3)
memory usage: 956.8+ KB


# Data



In [10]:
#External Test-1: https://github.com/Abdulk084/CardioTox/blob/master/data/external_test_set_pos.csv
ext_pos_df = pd.read_parquet(f"{dataset_path}/External-Data/pos-uniherg_db-gemini-embedding.parquet")

In [11]:
# External Test h70, h60 dataset: https://github.com/issararab/CToxPred/tree/main/data/raw/hERG
ext_h60_df = pd.read_parquet(f"{dataset_path}/External-Data/h60-uniherg_db-gemini-embedding.parquet")
ext_h70_df = pd.read_parquet(f"{dataset_path}/External-Data/h70-uniherg_db-gemini-embedding.parquet")

In [12]:
ext_h60_df.head()

,ACTIVITY,smiles,emb
0,0,CCOC(=O)C1(CCN(C)CC1)c1ccccc1,"[-0.0053594885, -0.030395241, -0.040858973, -0..."
1,0,CCN(CC)CC(=O)NC1=C(C)C=CC=C1C,"[-0.007807371, -0.025332695, -0.037532955, -0...."
2,0,CCCC(CCC)C(=O)O,"[0.016101563, -0.033382304, -0.032580573, -0.0..."
3,0,CCC(COC(=O)c1cc(OC)c(OC)c(OC)c1)(c1ccccc1)N(C)C,"[-0.0128125595, -0.011166253, -0.042656396, -0..."
4,0,COc1ccc(N(C(C)=O)c2cc3c(cc2[N+](=O)[O-])OC(C)(...,"[-0.01697734, 0.0033992645, -0.044581763, -0.0..."


In [13]:
!ls '{dataset_path}/External-Data'

CDK-external_test_set_pos.csv
CDK-herg60.csv
CDK-herg70.csv
eval_set_herg_60.csv
eval_set_herg_70.csv
external_test_set_pos.csv
h60-uniherg_db-deepseek-qwen1_5b-embedding.parquet
h60-uniherg_db-gemini-embedding.parquet
h70-uniherg_db-deepseek-qwen1_5b-embedding.parquet
h70-uniherg_db-gemini-embedding.parquet
herg_mmb_emb_external_test.npz
herg_mmb_emb_h60.npz
herg_mmb_emb_h70.npz
Morgan-external_test_set_pos.csv
Morgan-herg60.csv
Morgan-herg70.csv
pos-uniherg_db-deepseek-qwen1_5b-embedding.parquet
pos-uniherg_db-gemini-embedding.parquet


In [14]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X = ss.fit_transform(np.vstack(df['Embeddings'].values) )
y = df['classes']


In [15]:
X.shape,y.shape

((20409, 768), (20409,))

In [16]:
batch_size = 64
ds_train = tf.data.Dataset.from_tensor_slices((X,y)).batch(batch_size)
idx = np.random.random_integers(0,len(X),1000)
ds_test = tf.data.Dataset.from_tensor_slices((X[idx],y[idx])).batch(batch_size)

<ipython-input-16-21e067a5c8e3>:3: DeprecationWarning: This function is deprecated. Please call randint(0, 20409 + 1) instead
  idx = np.random.random_integers(0,len(X),1000)


# LTN

Predicate with softmax `P(x,class)`

In [17]:
class MLP(tf.keras.Model):
    """Model that returns logits."""
    def __init__(self, n_classes, hidden_layer_sizes=(16,16,8)):
        super(MLP, self).__init__()
        self.denses = [tf.keras.layers.Dense(s, activation="elu") for s in hidden_layer_sizes]
        self.dense_class = tf.keras.layers.Dense(n_classes)
        self.dropout = tf.keras.layers.Dropout(0.2)

    def call(self, inputs, training=False):
        x = inputs[0]
        for dense in self.denses:
            x = dense(x)
            x = self.dropout(x, training=training)
        return self.dense_class(x)

logits_model = MLP(2)
p = ltn.Predicate.FromLogits(logits_model, activation_function="softmax", with_class_indexing=True)

Constants to index/iterate on the classes

In [18]:
class_A = ltn.Constant(0, trainable=False)
class_B = ltn.Constant(1, trainable=False)
# class_C = ltn.Constant(2, trainable=False)

Operators and axioms

In [19]:
Not = ltn.Wrapper_Connective(ltn.fuzzy_ops.Not_Std())
And = ltn.Wrapper_Connective(ltn.fuzzy_ops.And_Prod())
Or = ltn.Wrapper_Connective(ltn.fuzzy_ops.Or_ProbSum())
Implies = ltn.Wrapper_Connective(ltn.fuzzy_ops.Implies_Reichenbach())
Forall = ltn.Wrapper_Quantifier(ltn.fuzzy_ops.Aggreg_pMeanError(p=2),semantics="forall")

In [20]:
formula_aggregator = ltn.Wrapper_Formula_Aggregator(ltn.fuzzy_ops.Aggreg_pMeanError(p=2))

@tf.function
def axioms(features, labels, training=False):
    x_A = ltn.Variable("x_A",features[labels==0])
    x_B = ltn.Variable("x_B",features[labels==1])
    # x_C = ltn.Variable("x_C",features[labels==2])
    axioms = [
        Forall(x_A,p([x_A,class_A],training=training)),
        Forall(x_B,p([x_B,class_B],training=training)),
        # Forall(x_C,p([x_C,class_C],training=training))
    ]
    for i in range(len(axioms)):
        if tf.math.is_nan(axioms[i].tensor):
            axioms[i].tensor  =0.0
    sat_level = formula_aggregator(axioms).tensor
    return sat_level

Initialize all layers and the static graph

In [21]:
for features, labels in ds_test:
    print("Initial sat level %.5f"%axioms(features,labels))
    break

Initial sat level 0.49483


# Training

Define the metrics. While training, we measure:
1. The level of satisfiability of the Knowledge Base of the training data.
1. The level of satisfiability of the Knowledge Base of the test data.
3. The training accuracy.
4. The test accuracy.

In [22]:
metrics_dict = {
    'train_sat_kb': tf.keras.metrics.Mean(name='train_sat_kb'),
    'test_sat_kb': tf.keras.metrics.Mean(name='test_sat_kb'),
    'train_accuracy': tf.keras.metrics.CategoricalAccuracy(name="train_accuracy"),
    'test_accuracy': tf.keras.metrics.CategoricalAccuracy(name="test_accuracy")
}

Define the training and test step

In [23]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
@tf.function
def train_step(features, labels):
    # sat and update
    with tf.GradientTape() as tape:
        sat = axioms(features, labels, training=True)
        loss = 1.-sat
    gradients = tape.gradient(loss, p.trainable_variables)
    optimizer.apply_gradients(zip(gradients, p.trainable_variables))
    sat = axioms(features, labels) # compute sat without dropout
    metrics_dict['train_sat_kb'](sat)
    # accuracy
    predictions = logits_model([features])
    metrics_dict['train_accuracy'](tf.one_hot(labels,2),predictions)

@tf.function
def test_step(features, labels):
    # sat
    sat = axioms(features, labels)
    metrics_dict['test_sat_kb'](sat)
    # accuracy
    predictions = logits_model([features])
    metrics_dict['test_accuracy'](tf.one_hot(labels,2),predictions)

Train

In [24]:
import commons

EPOCHS = 500

commons.train(
    EPOCHS,
    metrics_dict,
    ds_train,
    ds_test,
    train_step,
    test_step,
    csv_path="herg_gemini_results.csv",
    track_metrics=20
)

Epoch 0, train_sat_kb: 0.4803, test_sat_kb: 0.5264, train_accuracy: 0.5974, test_accuracy: 0.6460
Epoch 20, train_sat_kb: 0.5444, test_sat_kb: 0.5976, train_accuracy: 0.7425, test_accuracy: 0.7680
Epoch 40, train_sat_kb: 0.5728, test_sat_kb: 0.6160, train_accuracy: 0.7801, test_accuracy: 0.7820
Epoch 60, train_sat_kb: 0.5876, test_sat_kb: 0.6263, train_accuracy: 0.8006, test_accuracy: 0.8020
Epoch 80, train_sat_kb: 0.5994, test_sat_kb: 0.6474, train_accuracy: 0.8149, test_accuracy: 0.8260
Epoch 100, train_sat_kb: 0.6102, test_sat_kb: 0.6568, train_accuracy: 0.8278, test_accuracy: 0.8330
Epoch 120, train_sat_kb: 0.6183, test_sat_kb: 0.6637, train_accuracy: 0.8354, test_accuracy: 0.8430
Epoch 140, train_sat_kb: 0.6262, test_sat_kb: 0.6691, train_accuracy: 0.8434, test_accuracy: 0.8430
Epoch 160, train_sat_kb: 0.6318, test_sat_kb: 0.6730, train_accuracy: 0.8489, test_accuracy: 0.8580
Epoch 180, train_sat_kb: 0.6368, test_sat_kb: 0.6849, train_accuracy: 0.8531, test_accuracy: 0.8680
Epoch 

In [25]:
!ls "{dataset_path}/../Model-Weights"

hERG-Karim-CDK.keras	     hERG-UniChemDB-CDK.keras	    hERG-UniChemDB-Morgan_CDK.keras
hERG-Karim-MMB.keras	     hERG-UniChemDB-DeepSeek.keras  hERG-UniChemDB-Morgan.keras
hERG-Karim-Morgan_CDK.keras  hERG-UniChemDB-LLAMA.keras
hERG-Karim-Morgan.keras      hERG-UniChemDB-MMB.keras


In [26]:
logits_model.save(f"{dataset_path}/../Model-Weights/hERG-UniChemDB-gemini.keras")

## Model Evaluation

In [27]:
from sklearn.metrics import (
    accuracy_score as ays,
    f1_score as fs,
    precision_score as ps,
    recall_score as rs,
    matthews_corrcoef as mcc,
    roc_auc_score as auc,
    balanced_accuracy_score,
    confusion_matrix
)

In [28]:
def print_score(xtest,ytest,name):

    pred_test = logits_model.predict([xtest]).argmax(-1)

    auc_test = auc(ytest, pred_test)


    tn, fp, fn, tp = confusion_matrix(ytest, pred_test).ravel()

    specificity_test = tn / (tn + fp)

    sensitivity_test = tp / (tp + fn)

    NPV_test = tn / (tn + fn)

    PPV_test = tp / (tp + fp)
    Accuracy_test = ays(ytest, pred_test)
    Balanced_Accuracy_test = balanced_accuracy_score(ytest, pred_test)

    MCC_test= mcc(ytest, pred_test)


    print(f"MCC_test_{name}: " + str(MCC_test))
    print(f"NPV_test_{name}g: " + str(NPV_test))
    print(f"Accuracy_test_{name}: " + str(Accuracy_test))
    print(f"PPV_test_{name}: " + str(PPV_test))
    print(f"specificity_test_{name}: " + str(specificity_test))
    print(f"sensitivity_test_{name}: " + str(sensitivity_test))
    print(f"Balanced_Accuracy_test{name}: " + str(Balanced_Accuracy_test))


In [29]:
print_score(ss.transform(np.vstack(ext_pos_df['emb'].values)),ext_pos_df['ACTIVITY'],'External Data Test-1 (pos)')

2/2 [==============================] - 0s 6ms/step
MCC_test_External Data Test-1 (pos): 0.7008255578851517
NPV_test_External Data Test-1 (pos)g: 0.75
Accuracy_test_External Data Test-1 (pos): 0.8636363636363636
PPV_test_External Data Test-1 (pos): 0.9285714285714286
specificity_test_External Data Test-1 (pos): 0.8571428571428571
sensitivity_test_External Data Test-1 (pos): 0.8666666666666667
Balanced_Accuracy_testExternal Data Test-1 (pos): 0.861904761904762


In [30]:
print_score(ss.transform(np.vstack(ext_h60_df['emb'].values)),(ext_h60_df.ACTIVITY).astype(int),'External hERG-60')


2/2 [==============================] - 0s 5ms/step
MCC_test_External hERG-60: 0.7008255578851517
NPV_test_External hERG-60g: 0.75
Accuracy_test_External hERG-60: 0.8636363636363636
PPV_test_External hERG-60: 0.9285714285714286
specificity_test_External hERG-60: 0.8571428571428571
sensitivity_test_External hERG-60: 0.8666666666666667
Balanced_Accuracy_testExternal hERG-60: 0.861904761904762


In [31]:
print_score(ss.transform(np.vstack(ext_h70_df['emb'].values)),(ext_h70_df.pIC50 >=5).astype(int),'External hERG-70')


15/15 [==============================] - 0s 2ms/step
MCC_test_External hERG-70: 0.5340333148022405
NPV_test_External hERG-70g: 0.7076271186440678
Accuracy_test_External hERG-70: 0.7653276955602537
PPV_test_External hERG-70: 0.8227848101265823
specificity_test_External hERG-70: 0.7990430622009569
sensitivity_test_External hERG-70: 0.7386363636363636
Balanced_Accuracy_testExternal hERG-70: 0.7688397129186603
